# Mock MaNGA datacubes

### This notebook shows how to go from the synthetic iMaNGA datacubes, publicly available, to a mock MaNGA observation

#### It can be applied to any other syntethic datacube
#### It can be modified to obtain other mock observations


#### If used for scientific publications, please, cite Nanni+2022 for the datacubes, Nanni+2023a and Nanni+2023b for the iMaNGA VAC

In [ ]:

#!pip install spectral-cube

#!pip install reproject
#!pip install pyspeckit
#!pip install regions

In [6]:
import gc
import numpy as np
import matplotlib.pyplot as plt

import mplt_style
plt.style.use(mplt_style.style1)
import iMANGA_functions as iMANGA

from astropy.io import fits
from astropy.wcs import WCS # World coordinate system

from spectral_cube import SpectralCube    
from spectral_cube import LazyMask
import spectral_cube

import os

import copy


#rectangular cropping function

#%run "/media/ben/home/ben/Documents/Grad Stuff/MM data/Simplified/2025/0 Imports and functions.ipynb"
%run "0_Imports.ipynb"
# imports a function called Crop_Nans

In [8]:
"""
Select galaxy and===-------

Now we download the datacube for the galaxy ID=3 in snapshot=96, used in the first paper of the series as an example (https://academic.oup.com/mnras/article/515/1/320/6603844)


To download the MaNGA datacubes there are two options:


Download from the website through API

"""

#value added catalogue####################################################################################
VAC_path = "Vac_Path/iMaNGA_vac.fits"
VAC =  fits.open(VAC_path, mode='denywrite')
snap = 98 # running as .py you can pass it as an argument when running the script int(float(sys.argv[1]))
################################################################################################################
###########################################################################  
#        Dummy header
#####################
sc = SpectralCube.read("/media/ben/home/ben/Documents/Grad Stuff/MM data/Simplified/2025/Spectral Cubes/H13CN_J1-0.cube.fits")
header = copy.copy(sc.header)
########################################################
# Download simulation cubes and crop them to save space
########################################################
for i in range(818755,10000000):
    #seach for the galaxy fils in the catalogue
    try:
        galaxy_id_VAC = np.where(VAC[1].data['TNG_snap_id']==str(snap)+"-"+str(i))[0][0] # index of the galaxy in the iMaNGA VAC
    except Exception as e:
        #print(e,i)
        continue
    
    print("hit at",i)
    
    try:
        url = "http://www.tng-project.org/api/TNG50-1/snapshots/"+str(snap)+"/subhalos/"+str(i)+"/imanga.fits"
        r = iMANGA.get(url)
        datacube = fits.getdata(r)
        
        
        
        cube_name = str(snap)+"_"+str(i)+".fits"
        
        file = fits.open(cube_name)[0]
        

        ####################################################################################
        #make a fake pixel header so that WCS systems work
        
        header['NAXIS'] = 3
        header['NAXIS1'] = file.shape[2]
        header['NAXIS2'] = file.shape[1]
        header['NAXIS3'] = file.shape[0]
        
        header["CRPIX1"]  =  0
        header["CRPIX2"]  =  0
        header["CRPIX3"]  =  0
        header["CDELT1"]  =  1
        header["CDELT2"]  =  1
        header["CDELT3"]  =  1
        #These are just dummies############################
        header["CUNIT1"]  = 'deg'                
        header["CUNIT2"]  = 'deg'               
        header["CUNIT3"]  = 'm/s'               
        header["CTYPE1"]  = 'RA---SIN'          
        header["CTYPE2"]  = 'DEC--SIN'        
        header["CTYPE3"]  = 'VRAD'              
        ###################################################
        header["CRVAL1"]  =     0
        header["CRVAL2"]  =     0
        header["CRVAL3"]  =     0
        header["RESTFRQ"] =        0.0 # na
        #######################################################################################################################################################################
        ############################################################################################################################################
        fake_WCS = WCS(header) 
        
        # Create a Primary HDU (Header/Data Unit)
    
        hdu = fits.PrimaryHDU(file.data,header=header)
        
        # Create an HDU List
        hdul = fits.HDUList([hdu])
        
        # Save the FITS file
        
        cube_name = str(snap)+"_"+str(i)+"_h"+".fits"
        
        hdul.writeto(cube_name, overwrite=True)
        
        
        print("saved",cube_name)
        ############################
        ############################ Load the SC now that it has a WSC
        sc_SIM = SpectralCube.read(cube_name)
        
        
        #Mask the non-data to save sapce
        
        ##
        ## Fov grid
        ##
        grid_ =  VAC[3].data[3,galaxy_id_VAC,:,:] # FoV grid
        
        #
        # Mask data cube
        #
        
        grid, masked_data = iMANGA.apply_FOV(grid_, sc_SIM.hdu.data)
        #find mask edges
        print("Mask edges:")
        sx,ex,sy,ey = Crop_Nans(masked_data)
        
        scN = sc_SIM[:,sx-10:ex+10,sy-10:ey+10]
        
        cube_name = str(snap)+"_"+str(i)+"_h_c"+".fits"
        
        scN.write(cube_name)
        print("saved",cube_name)

        #cleanup
        del scN #remove interim cubes
        del sc_SIM
        del hdu
        del hdul
        del datacube 
        gc.collect()
        !mv $cube_name Sim_Cubes/
        
        
        
        #Remove header-fix cube and original cube to save space
        if os.path.exists(str(snap)+"_"+str(i)+"_h"+".fits"):
            os.remove(str(snap)+"_"+str(i)+"_h"+".fits")
            print("Deleted",str(snap)+"_"+str(i)+"_h"+".fits")
        if os.path.exists(str(snap)+"_"+str(i)+".fits"):
            os.remove(str(snap)+"_"+str(i)+".fits")
            print("Deleted",str(snap)+"_"+str(i)+".fits")
        
         
        print("done",i)
    except Exception as e:
        print(e,i)
        pass
    

hit at 818757
saved 98_818757_h.fits
Mask edges:


/tmp/ipykernel_957404/2585820252.py:13: RuntimeWarning: Mean of empty slice
  if(np.nanmean(data[0,lmi,:])>0 or np.nanmean(data[0,lmi,:])<0):
/tmp/ipykernel_957404/2585820252.py:18: RuntimeWarning: Mean of empty slice
  if(np.nanmean(data[0,:,lmj])>0 or np.nanmean(data[0,:,lmj])<0):
/tmp/ipykernel_957404/2585820252.py:22: RuntimeWarning: Mean of empty slice
  if(np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])>0 or np.nanmean(data[0,np.shape(data[0,:,:])[0]-lmi-1,:])<0):
/tmp/ipykernel_957404/2585820252.py:26: RuntimeWarning: Mean of empty slice
  if(np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])>0 or np.nanmean(data[0,:,np.shape(data[0,:,:])[1]-lmj-1])<0):


F 88
86 109 87 108
saved 98_818757_h_c.fits
Deleted 98_818757_h.fits
Deleted 98_818757.fits
done 818757


### We now input the FoV grid

In [ ]:
galaxy_id_VAC = np.where(VAC[1].data['TNG_snap_id']==str(snap)+"-"+str(gal))[0][0] # index of the galaxy in the iMaNGA VAC
##
## Fov grid
##
grid_ =  VAC[3].data[3,galaxy_id_VAC,:,:] # FoV grid
plt.imshow(grid_.T);
plt.title("FoV mask")

### Now we apply this mask to the datacube

In [ ]:
from astropy.visualization import simple_norm

fig = plt.figure(1, figsize=(6, 5))
fig.patch.set_facecolor('white')
ax=plt.subplot(111)
ax.minorticks_on()
ax.tick_params(length=8, which='major', direction="in", labelsize=26)
ax.tick_params(length=3, which='minor', direction="in", labelsize=26)


im = ax.imshow(FoV_datacube[300,:,:].T,cmap='gray'\
              , zorder=2,interpolation='nearest', origin='lower',\
               norm=simple_norm(FoV_datacube[3000,:,:].T, stretch='log', log_a=500))

cbar = plt.colorbar(im,ax=ax, pad = .0050, aspect=30)
cbar.ax.tick_params(labelsize=26, axis='y', direction='in', )

cbar.set_ticks([0.5*10**-17, 2*10**-17])


cbar.set_label(r'$ F_\lambda\quad\left[\,erg/\,s \,/m^2\,/\,\AA \,\,\right]$',fontsize=20);
cbar.ax.yaxis.get_offset_text().set_fontsize(24)


ax.set_xlim(94,200)
ax.set_ylim(94,200);

### Adding the noise 

As presented in Nanni+2022, we define the SNR as a function of the wavelength from MaNGA observation. 

The data are here available here. This is computed as the average of SNR functions for 100 randomly selected MaNGA galaxies, considering the spectra at around 1.5 Reff. Change the path accordingly. 

Including here another SNR function, other mocks can be created


In [ ]:
snr_file = np.genfromtxt("./iMaNGA_data/sdss_filters/snr_avarage.dat")

snr = snr_file[:,1]

In [ ]:
redshift = VAC[1].data[galaxy_id_VAC][2]

We here use the MaStar wavelegths, the file is available here: https://www.icg.port.ac.uk/mastar/

In [ ]:
ver = str("v0.2")
lib = str("th")

hdul = fits.open('./MaStar_SSP_'+ver+'.fits')
wave_rest=hdul[2].data[0,:] #lambda array
wave= wave_rest*(1+redshift)
hdul.close()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax = fig.gca()
ax.minorticks_on()
ax.tick_params(length=10, which='major', direction="in", labelsize=26)
ax.tick_params(length=5, which='minor', direction="in", labelsize=26)

plt.plot(wave, snr, linewidth=2, color = "#AF3133");

plt.ylabel('SNR', fontsize=26); 
plt.xlabel(r'$\lambda [\r{A}]$', fontsize=26);

#### Now, we can include the noise spaxel per spaxel, as in Eq. 4 in Nanni+2022

In [ ]:
grid_zero = grid.copy()
grid_zero[grid==True] = np.nanmin(FoV_datacube[:, (grid==True)],axis=0)>0

bkg_x = np.where((grid == True)&(grid_zero==True))[0][-1]
bkg_y = np.where((grid == True)&(grid_zero==True))[1][-1]

In [ ]:
bkg_  = FoV_datacube[:, bkg_x, bkg_y]

In [ ]:
pflux = np.zeros((FoV_datacube.shape))
newnoise = np.zeros((FoV_datacube.shape))

newnoise[:,:,:] = abs(np.sqrt(FoV_datacube[:,:,:])* (np.sqrt(bkg_.reshape(-1,1,1))/snr.reshape(-1,1,1)))
pflux[:,:,:] = np.random.normal(FoV_datacube[:,:,:], scale = newnoise[:,:,:])

In [ ]:
# to avoid unphysical noise produced by the random process 
pflux[np.less(pflux, 0., where=~np.isnan(pflux))] = np.min(pflux[np.greater(pflux, 0., where=~np.isnan(pflux))])
newnoise[np.less(newnoise, 0., where=~np.isnan(newnoise))] = np.min(newnoise[np.greater(newnoise, 0., where=~np.isnan(newnoise))])

#### Here it follows some examples of the effects of adding the noise to the spectra in different ares

### We can now add the effect of the point spread function

#### the point spread function, being added after the noise, make sure to reproduce the covariance among adjacent spacels in the datacube


#### we make use of the SDSS filters to convolve with the effective PSF in the differnt wavelegth band

##### this data are made available, change the path accordingly

In [ ]:
filer_g = np.genfromtxt('./iMaNGA_data/sdss_filters/SLOAN_SDSS.g.dat')
wave_g = filer_g[:,0]
response_g = filer_g[:,1]

filer_r = np.genfromtxt('./iMaNGA_data/sdss_filters/SLOAN_SDSS.r.dat')
wave_r = filer_r[:,0]
response_r = filer_r[:,1]

filer_i = np.genfromtxt('./iMaNGA_data/sdss_filters/SLOAN_SDSS.i.dat')
wave_i = filer_i[:,0]
response_i = filer_i[:,1]

filer_z = np.genfromtxt('./iMaNGA_data/sdss_filters/SLOAN_SDSS.z.dat')
wave_z = filer_z[:,0]
response_z = filer_z[:,1]


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax = fig.gca()
ax.minorticks_on()
ax.tick_params(length=10, which='major', direction="in", labelsize=26)
ax.tick_params(length=5, which='minor', direction="in", labelsize=26)

xc = int(np.shape(FoV_datacube[0,0,:])[0]/2)

plt.plot(wave, FoV_datacube[:, xc, xc], '-', alpha=.8, color='#010F13', linewidth=1.3, label='synthetic spectrum', zorder=3)
plt.plot(wave, pflux[:, xc, xc], '-', alpha=0.8, color='#AF3133', linewidth=1.3, label=r'perturbed', zorder=2)



plt.ylabel(r'$ F_\lambda\quad\left[\,erg/s \,/m^2/\,\AA \,\,\right]$', fontsize=26); 
plt.xlabel(r'$\lambda [\r{A}]$', fontsize=26) #micron
plt.legend(fontsize=26)
plt.grid(ls = '-.', lw = 0.55)
ax.yaxis.get_offset_text().set_fontsize(24)


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax = fig.gca()
ax.minorticks_on()
ax.tick_params(length=10, which='major', direction="in", labelsize=26)
ax.tick_params(length=5, which='minor', direction="in", labelsize=26)

xc = int(np.shape(FoV_datacube[0,0,:])[0]/2)

plt.plot(wave, FoV_datacube[:, xc-5, xc+5], '-', alpha=.8, color='#010F13', linewidth=1.3, label='synthetic spectrum', zorder=3)
plt.plot(wave, pflux[:, xc-5, xc+5], '-', alpha=0.8, color='#AF3133', linewidth=1.3, label=r'perturbed', zorder=2)



plt.ylabel(r'$ F_\lambda\quad\left[\,erg/s \,/m^2/\,\AA \,\,\right]$', fontsize=26); 
plt.xlabel(r'$\lambda [\r{A}]$', fontsize=26) #micron
plt.legend(fontsize=26)
plt.grid(ls = '-.', lw = 0.55)
ax.yaxis.get_offset_text().set_fontsize(24)

### We construct an average effective PSF in MaNGA observation (see Nanni+2022)
#### it is available to be used

In [ ]:
hdu_psf, headerPSF = fits.getdata("./iMaNGA_data/sdss_filters/ePSF.fits.gz", header=True)

psfg = hdu_psf[:,:, 0]
psfr = hdu_psf[:,:, 1]
psfi = hdu_psf[:,:, 2]
psfz = hdu_psf[:,:, 3]

##### Now we convolve the datacubes with the effective PSF

##### WARNING: it might takes a few minutes

In [ ]:
from astropy.convolution import convolve

In [ ]:
# We can resize the matrix for faster computation and lower computational cost when saving the files

In [ ]:
pflux_300 = pflux[:,~np.isnan(pflux[300,:,:]).all((1))]
pflux_300 = pflux_300[:,:,~np.isnan(pflux[300,:,:]).all((1))]

In [ ]:
grid = np.where(grid==0, np.nan,grid)
grid_300 = grid[:,~np.isnan(grid).all(1)]
grid_300 = grid_300[~np.isnan(grid).all(1)]

In [ ]:
im_convoluta = np.zeros((np.shape(pflux_300)))
for k in range(len(wave)):
    if wave[k]<=wave_r.min():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:], psfg), float("Nan"))
    if wave_r.min()<wave[k]<=wave_g.max():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:], np.mean((psfr, psfg), axis=0)), float("Nan"))
    if wave_g.max()<wave[k]<=wave_i.min():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:], psfr), float("Nan"))
    if wave_i.min()<wave[k]<=wave_r.max():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:], np.mean((psfr, psfi), axis=0)), float("Nan"))
    if wave_r.max()<wave[k]<=wave_z.min():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:],psfi), float("Nan"))
    if wave_z.min()<wave[k]<=wave_i.max():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:],np.mean((psfi, psfz), axis=0)), float("Nan"))
    if wave_i.max()<wave[k]<=wave_z.max():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:],psfz), float("Nan"))    
    if wave[k]>wave_z.max():
        im_convoluta[k, :,:] = np.where(grid_300==True, convolve(pflux_300[k, :,:],psfz), float("Nan"))    



In [ ]:
fig = plt.figure(1, figsize=(6, 5))
fig.patch.set_facecolor('white')
ax=plt.subplot(111)
ax.minorticks_on()
ax.tick_params(length=8, which='major', direction="in", labelsize=26)
ax.tick_params(length=3, which='minor', direction="in", labelsize=26)


im = ax.imshow(im_convoluta[300,:,:].T,cmap='gray'\
              , zorder=2,interpolation='nearest', origin='lower',\
               norm=simple_norm(im_convoluta[300,:,:].T, stretch='log', log_a=500))

cbar = plt.colorbar(im,ax=ax, pad = .0050, aspect=30)
cbar.ax.tick_params(labelsize=26, axis='y', direction='in', )

cbar.set_ticks([0.5*10**-17, 2*10**-17])


cbar.set_label(r'$ F_\lambda\quad\left[\,erg/\,s \,/m^2\,/\,\AA \,\,\right]$',fontsize=20);
cbar.ax.yaxis.get_offset_text().set_fontsize(24)

